## IMPORTS AND INSTALLS

Import all the needed libraries and installation of the least common ones for the correct functioning of the notebook

In [ ]:
#Install the needed packages

!pip install yfinance
!pip install mplfinance
!pip install pandas_ta
!pip install tscv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 914.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=51e62c94b766ca62e580d8c848184514bc0f26aa584f25c971113c8a312489bd
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [ ]:
#Import the needed packages

#Data handling
import numpy as np
import pandas as pd

#Financial packages
import yfinance as yf
import pandas_ta

#Charts
import matplotlib.pyplot as plt
import plotly.graph_objects as chart
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#Data processing and machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, balanced_accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import TimeSeriesSplit
from tscv import GapRollForward

## DATASET GENERATION

Retrieve the values from yahoo! finance and generate a dataset with the desired ones. Pre-process them in order to ease the dataset usage in further steps


In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf

#commodities = ['CC=F','CT=F','GC=F','HO=F','KC=F','KE=F','SB=F','ZC=F','ZL=F','ZO=F','ZS=F','ZR=F']
#names = ['cocoa','cotton','gold','heating_oil','coffee','wheat','sugar','corn','soybean_oil','oat','soybean','rough_rice']

commodities = ['CT=F','GC=F','HO=F','KC=F','NG=F','SB=F']
names = ['cotton','gold','heating_oil','coffee','natural_gas','sugar']

#Download the list, maximum available period and aggregate to access them by name
data = yf.download(tickers = commodities, period = 'max')
#Group by commodity instead of by characteristic
data.columns = data.columns.swaplevel(0, 1)
data.sort_index(axis=1, level=0, inplace=True)
#Rename columns
data.columns.set_levels(names,level=0, inplace=True)
#Subset to start after January 2010
data = data[data.index >= '2008-01-01 00:00:00']
data_prev = data[data.index <= '2020-01-01 00:00:00']
#Interpolate missing data
data = data.interpolate(limit = 3, limit_direction = 'forward')

data_adj_close = pd.DataFrame()
data_adj_close_prev = pd.DataFrame()
for name in names:
    data_adj_close[name] = data[name]['Adj Close']
    data_adj_close_prev[name] = data_prev[name]['Adj Close']

data_adj_close.to_csv('data_commodities.csv')

[*********************100%%**********************]  6 of 6 completed


<ipython-input-3-fe93c4550c50>:17: FutureWarning: inplace is deprecated and will be removed in a future version.
  data.columns.set_levels(names,level=0, inplace=True)


## DATASET PRE-PROCESSING

Definition of functions to process the original dataset and apply transformations, in order to generate the desired features and the target variable.

### Trend detection and encoding


*   Class definition through equal splits
*   Class definition according to a threshold



In [ ]:
#Split the dataset in n categories integrated by the same number of samples

def equal_classes(data,n_splits,window):
    #Extract the returns from data
    returns = data.pct_change(window).dropna()
    #Array for the labels
    labels = np.empty(len(data))
    labels[:] = np.nan
    #Split in equal classes
    sorted_returns = np.sort(returns,axis=0)
    splits = np.array_split(sorted_returns,n_splits)
    for i in range(len(returns)):
        val = returns.iloc[i]
        for j in range(len(splits)):
            if val in splits[j]:
                labels[i] = j

    return labels

In [ ]:
#Split the dataset in n categories according to the defined thresholds

def list_threshold(threshold,n_classes,append_inf):
    values = np.linspace(start=-threshold, stop=threshold, num = n_classes)
    if(append_inf):
        values = np.append(-np.inf,values)
        values = np.append(values,np.inf)
    return values

def encode_trend(p1,p2,threshold,n_classes):
    values = list_threshold(threshold,n_classes,True)
    change = p2/p1 - 1
    code = list_threshold(int(n_classes/2),n_classes,False)
    codified=0
    if change >= 0:
        codified = code[np.argmin(change > values) - 2]
    elif change < 0:
        codified = code[np.argmin(change > values) - 1]
    return codified


def trend_detection(data, window=2, n_classes=3, threshold=0.05):
    trend_result = 5*np.ones(len(data))
    for i in range(0,len(data)-window):
        trend_result[i] = encode_trend(data[i],data[i+window],threshold,n_classes)

    trend_result[trend_result == 5] = np.nan
    return trend_result

#### Features calculation and dataset generation

According to the previously defined methods, the classes are generated. Three types of trading indicators are calculated in order to add more information to the dataset:


*   Relative Strength Index (RSI)
*   Average Directional Index (ADX)
*   Simple Moving Average (SMA)






In [ ]:
#Trading indicators

def rsi(data,window=2):
    #Convert to dataframe to ease operation
    df = pd.DataFrame(data)
    #Calculate RSI for all samples
    df.ta.rsi(close=df.columns[0], length=window, append = True)

    return df.filter(regex = 'RSI').reset_index(drop = True)

def adx(data,window=2):
    #Convert to dataframe to ease operation
    df = pd.DataFrame(data).ta.adx(window).reset_index()

    return df.filter(regex = "ADX").reset_index(drop = True)

In [ ]:
def dataset_temporal(data_in, name_commodity, returns, rep, n_clas, window):

    #Calculate specified returns
    return_periods = pd.DataFrame()
    for val in returns:
        name = 'returns'+ str(val) + '_' + str(name_commodity)
        return_periods[name] = data_in.pct_change(val)

    #Shift returns along time to extract temporal dynamics
    data_temporal = pd.DataFrame()
    for i,ret in enumerate(returns):
        for j in range(rep):
            name = 'returns' + str(j*ret) + '_' + str((j+1)*ret) + '_' + str(name_commodity)
            data_temporal[name] = return_periods.iloc[:,i].shift(int(j*ret))

    df = data[name_commodity]

    for i, ret in enumerate(returns):
        name_rsi = 'RSI_' + str(ret)
        name_adx = 'ADX_' + str(ret)
        data_temporal[name_rsi] = df.ta.rsi(close=df['Adj Close'], length=ret)
        data_temporal[name_adx] = df.ta.adx(high=df['High'], low=df['Low'], close=df['Close'], length=ret).iloc[:,0]

    for i,ret in enumerate(returns):
        name = 'MA_' + str(ret)
        data_temporal[name] = df['Adj Close'].rolling(ret).mean()

    #Reset index to add trend without problems
    data_temporal = data_temporal.reset_index(drop=True)

    #Add trend to data
    name_trend = 'trend_' + str(name_commodity)
    data_temporal[name_trend] = equal_classes(data_in.reset_index(drop=True),n_clas,window)

    return data_temporal

#### Cross-validation methods

Two different cross-validation methods are tested:

*   GapRollForward. Acumulate past data to train the model.
*   BlockingTimeSeriesSplit. Train always with equally-sized blocks.



In [ ]:
# https://goldinlocks.github.io/Time-Series-Cross-Validation/

#Function to implement blocking cross-validation

class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits

    def get_n_splits(self, X, y, groups):
        return self.n_splits

    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.8 * (stop - start)) + start #Modified to get wanted splits
            yield indices[start: mid], indices[mid + margin: stop]

## MODEL EVALUATION



In [ ]:
def evaluate_dataset(data, smooth, sliding_window, n_features, n_splits, test_size, param_grid_rf, param_grid_svm, param_grid_mlp):
    if sliding_window:
        tscv = BlockingTimeSeriesSplit(n_splits = n_splits)
    else:
        tscv = GapRollForward(min_train_size = 1000, max_test_size=test_size, min_test_size=test_size - 50)

    if smooth:
        data = data.rolling(10).mean()

    data = data.dropna()

    X = data.iloc[:,range(n_features)]
    y = data.iloc[:,n_features].astype(int)

    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    print('Training observations: %d'% len(X_train))
    print('Testing observations: %d'% len(X_test))

    #Train a Random Forest Classifier
    rf = RandomForestClassifier()

    #Define parameter grid for crossvalidation
    param_grid = param_grid_rf

    grid_rf = GridSearchCV(rf,param_grid,scoring = 'accuracy', cv=tscv, n_jobs = -1,verbose = 3)
    grid_rf.fit(X_train,y_train)

    #Evaluate performance
    y_test_pred = grid_rf.predict(X_test)

    print(f"The accuracy of the Random Forest Classifier is: {accuracy_score(y_test, y_test_pred)}")
    print(f"\nThe confusion matrix for the Random Forest Classifier approach is: \n {confusion_matrix(y_test,y_test_pred)} ")

    #Classification report
    print(classification_report(y_test, y_test_pred))
    print(grid_rf.best_params_)

    #Show feature importances
    sort_importances = grid_rf.best_estimator_.feature_importances_.argsort()
    classifier = grid_rf.best_estimator_
    plt.rcParams["figure.figsize"] = (10,7)
    plt.barh(data.columns[sort_importances], grid_rf.best_estimator_.feature_importances_[sort_importances])

    print('\n\n')

    #Use selected variables from RF
    #X_train = X_train.iloc[:,sort_importances]
    #X_test = X_test.iloc[:,sort_importances]

    #X_train = X_train.iloc[:,0:5]
    #X_test = X_test.iloc[:,0:5]

    #Train a Support Vector Classifier
    steps = [('scaler',StandardScaler()),('SVM',SVC())]
    pipeline = Pipeline(steps)

    #Define parameter grid for crossvalidation
    param_grid = param_grid_svm

    grid_svc = GridSearchCV(pipeline,param_grid,scoring = 'accuracy', cv=tscv, n_jobs = -1,verbose = 3)
    grid_svc.fit(X_train,y_train)

    #Evaluate performance
    y_test_pred = grid_svc.predict(X_test)

    print(f"The accuracy of the Support Vector Classifier is: {accuracy_score(y_test, y_test_pred)}")
    print(f"\nThe confusion matrix for the Support Vector Classifier approach is: \n {confusion_matrix(y_test,y_test_pred)} ")

    #Classification report
    print(classification_report(y_test, y_test_pred))
    print(grid_svc.best_params_)

    print('\n\n')

    #Train a Multi Layer Perceptron
    steps = [('scaler',StandardScaler()),('MLP',MLPClassifier())]
    pipeline = Pipeline(steps)

    #Define parameter grid for crossvalidation
    param_grid = param_grid_mlp

    grid_mlp = GridSearchCV(pipeline,param_grid,scoring = 'accuracy', cv=tscv, n_jobs = -1,verbose = 3)
    grid_mlp.fit(X_train,y_train)

    #Evaluate performance
    y_test_pred = grid_mlp.predict(X_test)

    print(f"The accuracy of the Neural Network Classifier is: {accuracy_score(y_test, y_test_pred)}")
    print(f"\nThe confusion matrix for the Neural Network Classifier approach is: \n {confusion_matrix(y_test,y_test_pred)} ")

    #Classification report
    print(classification_report(y_test, y_test_pred))
    print(grid_mlp.best_params_)

    #Ensemble to create the voting classifier
    voting_clf = VotingClassifier(estimators=[("RandomForest", grid_rf.best_estimator_),
                                              ("SVC", grid_svc.best_estimator_),
                                              ("MLP", grid_mlp.best_estimator_)])
    voting_clf = voting_clf.fit(X_train,y_train)
    y_test_pred = voting_clf.predict(X_test)

    print(f"The accuracy of the Voting Classifier is: {accuracy_score(y_test, y_test_pred)}")
    print(f"\nThe confusion matrix for the Voting Classifier approach is: \n {confusion_matrix(y_test,y_test_pred)} ")

    #Classification report
    print(classification_report(y_test, y_test_pred))

## CODE EXECUTION

Usage of the previously defined functions with Natural Gas commodity

In [ ]:
#Moving Averages Display

df = data['natural_gas'].reset_index()
df['10ma'] = df['Adj Close'].rolling(window=10).mean()
df['100ma'] = df['Adj Close'].rolling(window=100).mean()
df['25ema'] = df['Adj Close'].ewm(25).mean()

fig = chart.Figure(data=[chart.Candlestick(x=df['Date'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'],
                name = "Cotton price")])

ma10_trace = chart.Scatter(x=df['Date'], y=df['10ma'], mode='lines', name='10MA', line=chart.scatter.Line(color="yellow"))
ma100_trace = chart.Scatter(x=df['Date'], y=df['100ma'], mode='lines', name='100MA', line=chart.scatter.Line(color="blue"))
ema25_trace = chart.Scatter(x=df['Date'], y=df['25ema'], mode='lines', name='25EMA', line=chart.scatter.Line(color="darkorange"))


fig.add_trace(ma10_trace)
fig.add_trace(ma100_trace)
fig.add_trace(ema25_trace)

fig.show()

In [ ]:
#Trading Indicators Display

df = df.reset_index()
df['80'] = [80] * len(df)
df['20'] = [20] * len(df)
df['25'] = [25] * len(df)

df['RSI'] = df.ta.rsi(close=df['Adj Close'], length=10)
df['ADX'] = df.ta.adx(high=df['High'], low=df['Low'], close=df['Close'], length=10).iloc[:,0]
df['BBU'] = df.ta.bbands(10).iloc[:,0]
df['BBL'] = df.ta.bbands(10).iloc[:,2]

fig = make_subplots(rows=3, cols=1, shared_xaxes=True,
               vertical_spacing=0.03, subplot_titles=('OHLC', 'RSI', 'ADX'),
               row_width=[0.15, 0.15, 0.55])

fig.add_trace(go.Candlestick(x=df['Date'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'],
                name = "OLHC"),row=1,col=1)
fig.add_trace(go.Line(x=df['Date'], y=df['BBL'], marker_color='navy', line = dict(width=1)
                     , name='Lower BB'), row=1, col=1)
fig.add_trace(go.Line(x=df['Date'], y=df['BBU'], marker_color='cornflowerblue', line = dict(width=1)
                     , name='Upper BB'), row=1, col=1)


fig.add_trace(go.Line(x=df['Date'], y=df['RSI'], showlegend=False, marker_color='black', line = dict(width=2)), row=2, col=1)
fig.add_trace(go.Line(x=df['Date'], y=df['80'], showlegend=False, marker_color='green', line = dict(width=1)), row=2, col=1)
fig.add_trace(go.Line(x=df['Date'], y=df['20'], showlegend=False, marker_color='red', line = dict(width=1)), row=2, col=1)

fig.add_trace(go.Line(x=df['Date'], y=df['ADX'], showlegend=False, marker_color='black', line = dict(width=2)), row=3, col=1)
fig.add_trace(go.Line(x=df['Date'], y=df['25'], showlegend=False, marker_color='blue', line = dict(width=1)), row=3, col=1)


fig.update(layout_xaxis_rangeslider_visible=False)
fig.show()

/usr/local/lib/python3.10/dist-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [ ]:
#Candlestick Chart

fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
               vertical_spacing=0.03, subplot_titles=('OHLC', 'Volume'),
               row_width=[0.2, 0.7])

fig.add_trace(go.Candlestick(x=df['Date'],
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'],
                name = "OLHC"),row=1,col=1)

fig.add_trace(go.Bar(x=df['Date'], y=df['Volume'], showlegend=False, marker_color='black'), row=2, col=1)

fig.update(layout_xaxis_rangeslider_visible=False)
fig.show()

In [ ]:
#Definition of the parameter grids for the different classifiers

param_grid_rf = {'max_depth': [6,12,18,24,30],
              'min_samples_split': [5,10,25,50],
              'n_estimators': [100,500,1000,5000],
              'criterion': ['gini','entropy'],
              'max_features': ['auto', 'sqrt', 'log2']}

param_grid_svm = {'SVM__C': [1e-7,1e-4,0.001,0.01,0.1,1,2,5],
              'SVM__gamma': [5,2,1,0.1,0.01,1e-4,1e-7],
              'SVM__kernel': ['linear','rbf','sigmoid']}

param_grid_mlp = {
        'MLP__hidden_layer_sizes': [(50,50,50), (50,100,50), (50,100,50,25)],
        'MLP__activation': ['tanh', 'relu'],
        'MLP__solver': ['sgd', 'adam'],
        'MLP__alpha': [0.0001, 0.001, 0.01, 0.1, 0.5], #check
        'MLP__learning_rate': ['constant','adaptive'],
        'MLP__max_iter':[500],
        'MLP__early_stopping': [True,False]
    }

In [ ]:
data_natural_gas = dataset_temporal(data_adj_close.natural_gas, 'natural_gas', [5,10,25,50], 6, 2, 21)
evaluate_dataset(data_natural_gas,smooth=False,sliding_window=True,n_features=36,n_splits=5,
                 test_size=200,param_grid_rf=param_grid_rf,param_grid_svm=param_grid_svm,param_grid_mlp=param_grid_mlp)

Training observations: 582
Testing observations: 146
Fitting 5 folds for each of 480 candidates, totalling 2400 fits
